In [33]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
import warnings
import re
warnings.filterwarnings("ignore")


In [73]:
base_url = "https://www.wissports.net/stats/tab_league_instance_player_stats/211430.js"
csrf_token = "xRU1eQ9QscQ0GM6oGPQ//8nzIINTwXV+14qa5X11A2+NDnD3jPItvNeGVm9iraNYKGd0H8xdQlq0eZ6AkNfNfQ=="
page_number = 3

# Define the query parameters based on your fetch command
params = {
    "dir": "desc",
    "order_by": "rshyd",
    "page": page_number,  # This is the key parameter for pagination
    "stat_module": "football_rushing",
    "subseason": "917849",
    "tag_match": "all",
    "tags[]": "1"
}

# Define the headers, including the essential X-CSRF-Token
headers = {
    "accept": "*/*;q=0.5, text/javascript, application/javascript, application/ecmascript, application/x-ecmascript",
    "x-csrf-token": csrf_token,  # Use the token you found in the network tab
    "x-requested-with": "XMLHttpRequest",
    # Adding a User-Agent is always a good idea
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

print(f"Fetching page {page_number}...")

response = requests.get(base_url, params=params, headers=headers)

Fetching page 3...


In [74]:
def remove_html_tags(text):
    clean = re.compile(r'<[^>]+>|// Page.+')
    return clean.sub('', text)

In [77]:
bs = BeautifulSoup(response.text)
raw_html = str(bs.find("table"))

In [79]:
html_strio = StringIO(raw_html.replace("\\n", "").replace('<\/t.+>', ''))

In [81]:
df = pd.read_html(html_strio)[0]
df.columns = list(pd.Series(df.columns).apply(remove_html_tags))
for col in df.columns:
    df[col] = df[col].apply(remove_html_tags)
df.iloc[-1, -1] = df.iloc[-1, -1].split(" ")[0]

In [83]:
df

,#,Name,Team,G,Att,Yds,Yds/Att,TD,FUM,2PM,Rush YPG
0,1,Paul Platts,CRAIG,10,64,465,7.3,2,0,0,46.5
1,8,Malik Blockman,MH,10,28,460,16.4,4,2,0,46.0
2,2,Nick Leto,MUS,12,69,451,6.5,5,3,0,37.6
3,2,Kaden Bates,OC,7,104,445,4.3,7,1,0,63.6
4,5,De'Jion Steele,MAD-W,8,69,437,6.3,5,2,0,54.6
5,34,Rashahn Hayes,MK,9,45,437,9.7,8,0,0,48.6
6,2,Steele Homme,HUD,5,64,430,6.7,7,1,0,86.0
7,33,Shaun Peeler,LAF,9,98,428,4.4,2,0,0,47.6
8,5,Damian Fisher,A-EAST,9,100,417,4.2,1,1,0,46.3
9,9,Tyler Konrad,SHB-N,5,36,413,11.5,4,0,0,82.6


In [ ]:
def read_wissports_page(page_number : int):
    